<a href="https://colab.research.google.com/github/peru-52/roster/blob/main/Chap8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade keras_nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.2 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [2]:
!pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [3]:
import os
os.environ["KERAS_BACKEND"]="tensorflow"
import keras
import tensorflow as tf
import numpy as np
from keras import layers

In [4]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=178028cca1bf25cb0668203f699a06a2062e76f47d3e799c96b5899ae3280a61
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [5]:
url = "https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip"

In [6]:
import wget
filename = wget.download(url)
filename

'ja.text8.zip'

In [7]:
!unzip ja.text8.zip

Archive:  ja.text8.zip
  inflating: ja.text8                


In [8]:
# utils.py
def load_data(filepath, encoding='utf-8'):
 with open(filepath, encoding=encoding) as f:
  return f.read()

In [9]:
# preprocessing.py
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams, make_sampling_table

def build_vocabulary(text, num_words=None):
 tokenizer = Tokenizer(
    num_words=num_words,
    oov_token='<unk>'
 )
 tokenizer.fit_on_texts([text])
 return tokenizer

def create_dataset(text, vocab, num_words, window_size,negative_samples):
 data = vocab.texts_to_sequences([text]).pop()
 sampling_table = make_sampling_table(num_words)

 couples, labels = skipgrams(data, num_words,
                             window_size=window_size,
                             negative_samples=negative_samples,
                             sampling_table=sampling_table)
 word_target, word_context = zip(*couples)
 word_target = np.reshape(word_target, (-1,1))
 word_context = np.reshape(word_context, (-1,1))
 labels = np.asarray(labels)
 return [word_target, word_context], labels

In [10]:
# model.py
from keras.models import Model
from keras.layers import Input, Dot, Flatten, Embedding, Dense

In [11]:
class EmbeddingModel:
 def __init__(self, vocab_size, emb_dim=100):
    self.word_input = Input(shape=(1,), name='word_input')
    self.word_embed = Embedding(input_dim=vocab_size,
                                output_dim=emb_dim,
                                input_length=1,
                                name='word_embedding')
    self.context_input = Input(shape=(1,), name='context_input')
    self.context_embed = Embedding(input_dim=vocab_size,
                                   output_dim=emb_dim,
                                   input_length=1,
                                   name='context_embedding')
    self.dot = Dot(axes=2)
    self.flatten = Flatten()
    self.output = Dense(1, activation='sigmoid')

 def build(self):
    word_embed = self.word_embed(self.word_input)
    context_embed = self.context_embed(self.context_input)
    dot = self.dot([word_embed, context_embed])
    flatten = self.flatten(dot)
    output = self.output(flatten)
    model = Model(inputs=[self.word_input, self.context_input],
                  outputs=output)
    return model

In [12]:
# inference.py
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

class InferenceAPI:
  def __init__(self, model, vocab):
      self.vocab = vocab
      self.weights = model.get_layer('word_embedding').get_weights()[0]

  def most_similar(self, word, topn=10):
      word_index = self.vocab.word_index.get(word, 1)
      sim = self._cosine_similarity(word_index)
      pairs = [(s, i) for i, s in enumerate(sim)]
      pairs.sort(reverse=True)
      pairs = pairs[1: topn + 1]
      res = [(self.vocab.index_word[i], s) for s, i in pairs]
      return res

  def similarity(self, word1, word2):
      word_index1 = self.vocab.word_index.get(word1, 1)
      word_index2 = self.vocab.word_index.get(word2, 1)
      weight1 = self.weights[word_index1]
      weight2 = self.weights[word_index2]
      return cosine(weight1, weight2)

  def _cosine_similarity(self, target_index):
      target_weight = self.weights[target_index]
      similarity = cosine_similarity(self.weights, [target_weight])
      return similarity.flatten()

In [13]:
from pprint import pprint
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

In [14]:
emb_dim = 50
epochs = 10
model_path = 'model.keras'
negative_samples = 1
num_words = 10000
window_size = 1

In [15]:
text = load_data(filepath='ja.text8')

vocab = build_vocabulary(text, num_words)

x, y = create_dataset(text, vocab, num_words,
                      window_size, negative_samples)

model = EmbeddingModel(num_words, emb_dim)
model = model.build()
model.compile(optimizer='adam', loss='binary_crossentropy')

callbacks = [
 EarlyStopping(patience=1),
 ModelCheckpoint(model_path, save_best_only=True)
]

model.fit(x=x,
          y=y,
          batch_size=128,
          epochs=epochs,
          validation_split=0.2,
          callbacks=callbacks)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['word_input', 'context_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


59582/59582 ━━━━━━━━━━━━━━━━━━━━ 149s 2ms/step - loss: 0.2884 - val_loss: 0.2030
Epoch 2/10
59582/59582 ━━━━━━━━━━━━━━━━━━━━ 148s 2ms/step - loss: 0.1855 - val_loss: 0.1862
Epoch 3/10
59582/59582 ━━━━━━━━━━━━━━━━━━━━ 201s 2ms/step - loss: 0.1661 - val_loss: 0.1789
Epoch 4/10
59582/59582 ━━━━━━━━━━━━━━━━━━━━ 145s 2ms/step - loss: 0.1558 - val_loss: 0.1750
Epoch 5/10
59582/59582 ━━━━━━━━━━━━━━━━━━━━ 202s 2ms/step - loss: 0.1487 - val_loss: 0.1733
Epoch 6/10
59582/59582 ━━━━━━━━━━━━━━━━━━━━ 147s 2ms/step - loss: 0.1433 - val_loss: 0.1712
Epoch 7/10
59582/59582 ━━━━━━━━━━━━━━━━━━━━ 145s 2ms/step - loss: 0.1391 - val_loss: 0.1699
Epoch 8/10
59582/59582 ━━━━━━━━━━━━━━━━━━━━ 202s 2ms/step - loss: 0.1363 - val_loss: 0.1711


In [16]:
from pprint import pprint
import numpy as np

# モデルとAPIのロード
model = load_model('model.keras')  # 学習済みモデルのロード
api = InferenceAPI(model, vocab)  # 類似度計算APIのインスタンス作成

# 5つの単語に共通する類似単語を取得する関数
def get_common_similar_words(api, words, topn=10):
    try:
        # 各単語の埋め込みを取得
        embeddings = []
        for word in words:
            word_index = api.vocab.word_index.get(word, None)
            if word_index is not None:
                embeddings.append(api.weights[word_index])
            else:
                print(f"単語 '{word}' はボキャブラリに存在しません。無視します。")

        if not embeddings:
            print("有効な単語が入力されていません。")
            return

        # ベクトルの平均を計算
        average_embedding = np.mean(embeddings, axis=0)

        # 平均ベクトルに基づいて類似単語を取得
        similarity = cosine_similarity(api.weights, [average_embedding]).flatten()
        most_similar_indices = similarity.argsort()[-topn-1:][::-1][1:]  # 自身を除外

        # 類似単語を取得
        result = [(api.vocab.index_word[idx], similarity[idx]) for idx in most_similar_indices]
        pprint(result)

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# ユーザー入力
print("5つの単語を入力してください（スペース区切り）。")
input_words = input("入力: ").strip().split()

if len(input_words) == 5:  # 5つの単語が入力された場合のみ実行
    get_common_similar_words(api, input_words, topn=10)
else:
    print("エラー: 必ず5つの単語を入力してください！")

5つの単語を入力してください（スペース区切り）。
入力: 猫 犬 狼 豚 羊
[('猫', 0.9204491),
 ('羊', 0.90933365),
 ('蛇', 0.9058565),
 ('クモ', 0.8949733),
 ('鳥', 0.88749444),
 ('爪', 0.88749),
 ('狼', 0.8844025),
 ('影', 0.87872165),
 ('夢', 0.8773711),
 ('破', 0.87658095)]


In [18]:
from pprint import pprint

# モデルとAPIのロード
model = load_model('model.keras')  # 学習済みモデルのロード
api = InferenceAPI(model, vocab)  # 類似度計算APIのインスタンス作成

# 類似単語を取得する関数
def display_similar_words(api, words, topn=10):
    for word in words:
        print(f"\n【単語】: {word}")
        try:
            similar_words = api.most_similar(word=word, topn=topn)
            pprint(similar_words)  # 類似単語を出力
        except KeyError:
            print(f"単語 '{word}' はボキャブラリに存在しません。")

# ユーザー入力
print("5つの単語を入力してください（スペース区切り）。")
input_words = input("入力: ").strip().split()

if len(input_words) == 5:  # 5つの単語が入力された場合のみ実行
    display_similar_words(api, input_words, topn=10)
else:
    print("エラー: 必ず5つの単語を入力してください！")

5つの単語を入力してください（スペース区切り）。
入力: 猫 犬 狼 豚 羊

【単語】: 猫
[('唄', 0.8827321),
 ('蛇', 0.87987155),
 ('天井', 0.8657071),
 ('我', 0.85345125),
 ('鳥', 0.8534021),
 ('皮', 0.8444024),
 ('爪', 0.84416175),
 ('精', 0.8431183),
 ('ウサギ', 0.84307057),
 ('クモ', 0.84300303)]

【単語】: 犬
[('豚', 0.8063334),
 ('恐竜', 0.79668355),
 ('宇宙船', 0.78453475),
 ('クモ', 0.7818955),
 ('物体', 0.7784943),
 ('部屋', 0.77039504),
 ('洞窟', 0.7569414),
 ('銀', 0.7540104),
 ('事典', 0.7538121),
 ('爪', 0.7530182)]

【単語】: 狼
[('幻', 0.8762877),
 ('姫', 0.8597362),
 ('雪', 0.85711294),
 ('蛇', 0.8565743),
 ('恋', 0.85608166),
 ('剣', 0.85352784),
 ('太夫', 0.8530679),
 ('かれ', 0.8485142),
 ('霧', 0.84840065),
 ('不動', 0.8475653)]

【単語】: 豚
[('猫', 0.8424875),
 ('魚', 0.8104037),
 ('鳥', 0.80754024),
 ('犬', 0.8063334),
 ('皮', 0.8027852),
 ('棺', 0.8015492),
 ('羊', 0.8000477),
 ('涙', 0.7998967),
 ('クモ', 0.79908043),
 ('天井', 0.79722905)]

【単語】: 羊
[('母', 0.85534763),
 ('蛇', 0.85423636),
 ('張', 0.84902734),
 ('鹿', 0.8472137),
 ('雪', 0.84695554),
 ('蝶', 0.84272975),
 ('紫'

#考察
今回は動物の単語を入れたため、同じように動物を表す単語が出力されると考えた。しかし、実際には動物とは関係がない単語も表示された。動物と関連強い単語が出力されていると考えた。しかし、動物に関連する単語の出力もあった。爪や皮などは動物を形成する一部分を表す単語である。「猫」が一番最初に入力されているため、猫が与える影響が大きいと考えた。
単語ごとだと、より入力された単語から連想されやすい単語が出力されていると考えた。